# Environment Preparation

## Install the Environment

In [34]:
!pip install requirements.txt

## Import the Environment

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import requests

# declear and import data
mp_drawing =mp.solutions.drawing_utils
mp_drawing_styles =mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

## Parameter Define & initialize

In [23]:
#record all the 33 different key landmarks in each frame of the video
dir1={}
#Key for the video parameter
flag=0
#Count how many fall frame happened
counter = 0
#Initialize the body angle to the camera
body_angle='front'

#body position parameter
sideway_slight=0
sideway_whole=0
front = 0

#Fall parameters
fall=0

## Video Import

In [24]:
#the path of video to import
video_path = r'C:\Users\22095\Desktop\Third year project\UR Fall Detection Dataset\Normal.mp4'
cap = cv2.VideoCapture(video_path)

## Real-time video import

In [ ]:
#Depend on which camera device we are using
#0 is main camera; 1 is the external camera 
cap = cv2.VideoCapture(1)

# Fallen and Falling detection

In [25]:
with mp_pose.Pose(min_detection_confidence=0.6,min_tracking_confidence=0.6) as pose:
    while cap.isOpened():
    #while True:
        success,image = cap.read()
        if image is None:
            break
        if not success:
            print('Ignoring empty camera frame')
            #loading a video, use break
            #real-time, use continue
            break
           #continue
        #To improve the performance, change the image to note writeable
        image.flags.writeable = False
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        results = pose.process(image)
        
        #Make a judgement to see whether recongized the landmark
        lst=[]
        p_lst = results.pose_landmarks
        #check whether there is an input
        if p_lst is None:
            print('no point is founded')
            continue
        else:
            for i in results.pose_landmarks.landmark:
                lst.append((i.x, i.y, i.z, i.visibility))
        h, w, _ = image.shape
            

 # Parameter settings
        ##  设肩宽为38 上半身60 躯干的比例为0.6
        ##  因为有可能弯腰，肩胯长不该是肩与胯的高度之差，但由于无法精确地确定身体的角度，无法准确地求出上半身的长度
        shoulder_wide = abs(lst[11][0] - lst[12][0])
        print(lst[23][1],lst[24][1],'\t',lst[11][1],lst[12][1])
        
        #the height of the shoulder to hip
        s_h_high = abs((lst[23][1]+lst[24][1]-lst[11][1]-lst[12][1])/2)

        #the length between the shoudler and the hip
        s_h_long=np.sqrt(((lst[23][1]+lst[24][1]-lst[11][1]-lst[12][1] )/2)**2+((lst[23][0]+lst[24][0]-lst[11][0]-lst[12][0] )/2)**2)

        #the height of the hip to feet
        h_f_high = ((lst[28][1]+lst[27][1]-lst[24][1]-lst[23][1])/2)
         
        #the length between the hip and the feet
        #这是在两只脚与肩膀同宽的情况下，如果可以需要把这个设为定值
        h_f_long=np.sqrt(((lst[28][1]+lst[27][1]-lst[24][1]-lst[23][1] )/2)**2+((lst[28][0]+lst[27][0]-lst[24][0]-lst[23][0] )/2)**2)
        
    #Parameters for avoiding bow detect as fall
    #=========================================================================
        #the length between hip and the palm which is the ground
        h_g_high = abs((lst[23][1] + lst[24][1] - lst[29][1] - lst[30][1])/2)
        
        h_g_long=np.sqrt(((lst[32][1]+lst[31][1]-lst[24][1]-lst[23][1] )/2)**2+((lst[32][0]+lst[27][0]-lst[31][0]-lst[23][0] )/2)**2)
        
        #the length between shoulder and the ground
        s_g_high = abs((lst[11][1] + lst[12][1] - lst[29][1] - lst[30][1])/2)
    #=========================================================================    
        
        rate1=shoulder_wide/s_h_high

# determine the orientation to the camera
# the orientation to the camera is crucial that becouse usually we need to determine the orientation has a huge influence to the fallen posture
        if 0.2< rate1 < 0.4:
            sideway_slight +=1
            sideway_whole = 0
            front = 0
        elif rate1< 0.2 :
            sideway_whole+=1
            sideway_slight=0
            front = 0
        else:
            sideway_whole  = 0
            sideway_slight = 0
            front = 0
        if sideway_slight >= 3:
            print(f'sideway slight')
            sideway_slight = 0
            body_angle = 'sideway slight'
        elif sideway_whole >= 3:
            print(f'sideway whole')
            sideway_whole = 0
            body_angle = 'sideway whole'
        else:
            front += 1
        if front >= 3:
            body_angle = 'front'
            front = 0
            print('front')
            
                        
        print('s_h_high: ',s_h_high, 's_h_long: ',s_h_long,'h_f_high: ', h_f_high,'h_f_long: ')


#Fall detection algorithm parameters
#==================================================================================
        para_s_h_1 = 1.15
        para_s_h_2 = 0.85
        para_h_f = 0.6
        para_fall_time = 5

#Fall detection Step 1
#first Part test code for detect Not Fall
#==================================================================================
        if s_h_high < s_h_long*para_s_h_1 and s_h_high > s_h_long*para_s_h_2:
            print(f'Not Fall')
            fall = 0

#Fall detection Step 2
#==================================================================================
        elif h_f_high < para_h_f * h_f_long:
            fall+=1
        else:
            fall=0
            print(f'Bend Over')
        if fall>=para_fall_time:
            print(f'fall')
            counter += 1
            
           #Call Push Deer's API to use Push Deer's notification push function
           #api="https://api2.pushdeer.com/message/push?pushkey=<the key of mobile device> & text= Fall  "
           #req = requests.post(api)

            fall=0
            print(lst[0][1],'\t',lst[11][1],'\t',lst[23][1])
        print('============================================================================================================')           
            
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                 )    

#Falling detection
#Different frame comparation
#================================================================================== 
        #contain frames with 33 different keypoints information
        dir1[str(flag)]=(lst)
        #time parameter
        flag+=1
        

        if len(dir1) % 30 == 0:
            #choose frames in a certain constant interval to include more information
            interval = 1
            time_num = len(dir1)//interval
            #Calculate landmarks in each second
            start_num = len(dir1) - 30
            #initialize the falling counter
            falling1 = 0
            #test
            print(time_num)

            for i in range(start_num+6 ,time_num):

#Falling detection algorithm parameters
#==================================================================================
                now_lst = dir1[str(i*interval)]
                pre_lst = dir1[str((i-6)*interval)]

                s_h_high = (pre_lst[23][1] - pre_lst[11][1] + pre_lst[24][1] - pre_lst[12][1])/2
                s_h_long=np.sqrt(((pre_lst[23][1]+pre_lst[24][1]-pre_lst[11][1]-lst[12][1] )/2)**2+((pre_lst[23][0]+pre_lst[24][0]-pre_lst[11][0]-pre_lst[12][0] )/2)**2)

                para_falling_s_h_1 = 1.15
                para_falling_s_h_2 = 0.85
                para_v_1 = 0.5
                para_v_2 = 0.3
                para_falling_time = 3
                


#Falling detection Step 1
#First Part test code for detect Not Falling
#==================================================================================
                if s_h_high < s_h_long*para_falling_s_h_1 and s_h_high > s_h_long*para_falling_s_h_2:
                    print(f'not falling')
                elif now_lst[0][1] < para_v_1*((pre_lst[11][1] + pre_lst[12][1])/2):
                    print(f'falling step 1')
                    falling1 +=1

                #     if now_lst[0][1] < para_v_2*((pre_lst2[11][1] + pre_lst2[12][1])/2):
                #         print(f'falling step 2')
                #         falling2 +=1

                    if falling1 >= para_falling_time:

                        print(f'falling situation now')
                        #refresh falling counter
                        falling1= 0
                

                        #Call Push Deer's API to use Push Deer's notification push function
                        #api="https://api2.pushdeer.com/message/push?pushkey=<the key of mobile device> & text= Falling  "
                        #req = requests.post(api)
                        
                else:
                    print(f'not falling in this second')
                    #refresh falling counter
                    falling1=0
            #release the space of dictionary
            dic1 = {}

              
        

        #setup status box
        cv2.rectangle(image,(0,0),(225,130),(245,117,16),-1)
        
        # rep data
        cv2.putText(image,'Fall Frames Number',(15,12),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,'Body Angle',(15,80),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
        cv2.putText(image,str(counter),(10,65),
                     cv2.FONT_HERSHEY_SIMPLEX,2,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(image,str(body_angle),(10,110),
                     cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2,cv2.LINE_AA)

                    
                
        cv2.imshow('Mediapipe Feed', image)
            
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    


no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded
no point is founded


test
0.46644049882888794 0.46974414587020874 	 0.2482796013355255 0.27681413292884827
s_h_high:  0.20554545521736145 s_h_long:  0.20576697080804057 h_f_high:  0.3004603981971741 h_f_long: 
Not Fall
26
test
0.46817439794540405 0.4702712595462799 	 0.24968141317367554 0.2749626040458679
sideway whole
s_h_high:  0.20690082013607025 s_h_long:  0.20712946422989192 h_f_high:  0.30444832146167755 h_f_long: 
Not Fall
27
test
0.4680355191230774 0.4701070487499237 	 0.24974727630615234 0.274655818939209
s_h_high:  0.20686973631381989 s_h_long:  0.20713414656745233 h_f_high:  0.3050182908773422 h_f_long: 
Not Fall
28
test
0.46768659353256226 0.46928563714027405 	 0.24950480461120605 0.27150601148605347
s_h_high:  0.2079807072877884 s_h_long:  0.20843806468436282 h_f_high:  0.30817295610904694 h_f_long: 
Not Fall
29
test
0.46760809421539307 0.46905046701431274 	 0.24988330900669098 0.27260875701904297
sideway whole
s_h_high:  0.20708324760198593 s_h_long:  0.20773876362476174 h_f_high:  0.31162002

test
0.5903298258781433 0.59302818775177 	 0.43291911482810974 0.4714798331260681
s_h_high:  0.13947953283786774 s_h_long:  0.1489404952902743 h_f_high:  0.19496256113052368 h_f_long: 
Not Fall
56
test
0.6021568775177002 0.6018134951591492 	 0.44340780377388 0.47833675146102905
sideway whole
s_h_high:  0.14111290872097015 s_h_long:  0.15048743066534528 h_f_high:  0.17845019698143005 h_f_long: 
Not Fall
57
test
0.6103336811065674 0.613449215888977 	 0.4519699215888977 0.4947015643119812
s_h_high:  0.13855570554733276 s_h_long:  0.1481494022406064 h_f_high:  0.16969436407089233 h_f_long: 
Not Fall
58
test
0.6239535808563232 0.625177800655365 	 0.4595736861228943 0.5061719417572021
s_h_high:  0.1416928768157959 s_h_long:  0.15022462212899368 h_f_high:  0.15623369812965393 h_f_long: 
Not Fall
59
test
0.6308519840240479 0.6318250298500061 	 0.4672727584838867 0.5148107409477234
sideway whole
s_h_high:  0.14029675722122192 s_h_long:  0.1483451927101347 h_f_high:  0.14970415830612183 h_f_long

test
0.7280111908912659 0.7441293597221375 	 0.5301968455314636 0.582234263420105
sideway whole
s_h_high:  0.17985472083091736 s_h_long:  0.1829857220910711 h_f_high:  0.03042694926261902 h_f_long: 
Not Fall
81
test
0.7294224500656128 0.7436573505401611 	 0.532119631767273 0.5817822813987732
s_h_high:  0.1795889437198639 s_h_long:  0.18264072533149409 h_f_high:  0.031007111072540283 h_f_long: 
Not Fall
82
test
0.7304928302764893 0.7421558499336243 	 0.5331564545631409 0.5820976495742798
s_h_high:  0.17869728803634644 s_h_long:  0.18189732263899147 h_f_high:  0.03362366557121277 h_f_long: 
Not Fall
83
test
0.7308013439178467 0.740985095500946 	 0.5334815979003906 0.5837764739990234
sideway whole
s_h_high:  0.17726418375968933 s_h_long:  0.18057916412445857 h_f_high:  0.03853952884674072 h_f_long: 
Not Fall
84
test
0.7311237454414368 0.7406826615333557 	 0.5344223380088806 0.5835846066474915
s_h_high:  0.1768997311592102 s_h_long:  0.18025074142837297 h_f_high:  0.04055723547935486 h_f_l

test
0.486068993806839 0.4846216142177582 	 0.29469743371009827 0.30652421712875366
s_h_high:  0.18473447859287262 s_h_long:  0.18475002468816437 h_f_high:  0.23329263925552368 h_f_long: 
Not Fall
105
test
0.48379629850387573 0.4812159538269043 	 0.2924915850162506 0.30241915583610535
s_h_high:  0.18505075573921204 s_h_long:  0.18506939229294786 h_f_high:  0.23176395893096924 h_f_long: 
Not Fall
106
test
0.4827987253665924 0.480989009141922 	 0.2899855673313141 0.2997451424598694
sideway slight
s_h_high:  0.18702851235866547 s_h_long:  0.18704737649983869 h_f_high:  0.23591893911361694 h_f_long: 
Not Fall
107
test
0.48329535126686096 0.4813079237937927 	 0.28875982761383057 0.2968938648700714
s_h_high:  0.18947479128837585 s_h_long:  0.18950975143266596 h_f_high:  0.2377181202173233 h_f_long: 
Not Fall
108
test
0.4830901324748993 0.48096033930778503 	 0.28675296902656555 0.29437267780303955
s_h_high:  0.1914624124765396 s_h_long:  0.19148342560744547 h_f_high:  0.26507535576820374 h_f_

...

test
0.7906967997550964 0.7969826459884644 	 0.78190016746521 0.7778574228286743
s_h_high:  0.013960927724838257 s_h_long:  0.014894306101244089 h_f_high:  0.0478532612323761 h_f_long: 
Not Fall
4615
test
0.7925750613212585 0.7986882925033569 	 0.77993243932724 0.7787758708000183
s_h_high:  0.01627752184867859 s_h_long:  0.01696856412662578 h_f_high:  0.046338677406311035 h_f_long: 
Not Fall
4616
test
0.793510377407074 0.7981260418891907 	 0.7811306118965149 0.7824625372886658
s_h_high:  0.014021635055541992 s_h_long:  0.01479961709092357 h_f_high:  0.04795113205909729 h_f_long: 
Not Fall
4617
test
0.8003280758857727 0.8047759532928467 	 0.7836273312568665 0.7875196933746338
s_h_high:  0.01697850227355957 s_h_long:  0.017466175336892563 h_f_high:  0.04400661587715149 h_f_long: 
Not Fall
4618
test
0.7998475432395935 0.8030535578727722 	 0.7876635789871216 0.7902613878250122
s_h_high:  0.012488067150115967 s_h_long:  0.01275536546153179 h_f_high:  0.0387139618396759 h_f_long: 
Not Fall
4

test
0.8487669229507446 0.8500576615333557 	 0.8353435397148132 0.8433486819267273
s_h_high:  0.010066181421279907 s_h_long:  0.010519773027894412 h_f_high:  0.014733701944351196 h_f_long: 
Not Fall
4640
test
0.8444202542304993 0.8494223356246948 	 0.8277538418769836 0.8388627767562866
s_h_high:  0.013612985610961914 s_h_long:  0.013888668705196434 h_f_high:  0.018073558807373047 h_f_long: 
Not Fall
4641
test
0.8611171245574951 0.8685638308525085 	 0.8303759694099426 0.8393186926841736
s_h_high:  0.029993146657943726 s_h_long:  0.030123927208609888 h_f_high:  0.02494528889656067 h_f_long: 
Not Fall
4642
test
0.8592346906661987 0.866550862789154 	 0.8227846026420593 0.8402329683303833
s_h_high:  0.03138399124145508 s_h_long:  0.03224960482493309 h_f_high:  0.02977234125137329 h_f_long: 
Not Fall
4643
test
0.8648788332939148 0.8771123290061951 	 0.8230645656585693 0.8466670513153076
s_h_high:  0.036129772663116455 s_h_long:  0.03665716121990846 h_f_high:  0.02707761526107788 h_f_long: 
N

test
0.867885947227478 0.8791736364364624 	 0.8796346783638 0.9135950207710266
s_h_high:  0.023085057735443115 s_h_long:  0.025090740048891898 h_f_high:  0.03381246328353882 h_f_long: 
Not Fall
4665
test
0.869968056678772 0.8819111585617065 	 0.8836562633514404 0.9185882210731506
s_h_high:  0.025182634592056274 s_h_long:  0.027442341507539495 h_f_high:  0.035694241523742676 h_f_long: 
Not Fall
4666
test
0.8730136156082153 0.8851032257080078 	 0.8827663660049438 0.916674792766571
s_h_high:  0.020662158727645874 s_h_long:  0.023266795062547908 h_f_high:  0.03857612609863281 h_f_long: 
Not Fall
4667
test
0.8729730248451233 0.8848896622657776 	 0.884225070476532 0.9145659804344177
s_h_high:  0.020464181900024414 s_h_long:  0.022717254288665785 h_f_high:  0.041940122842788696 h_f_long: 
Not Fall
4668
test
0.8716245293617249 0.88273686170578 	 0.8837377429008484 0.9126595258712769
s_h_high:  0.02101793885231018 s_h_long:  0.023220640863017745 h_f_high:  0.043693095445632935 h_f_long: 
Not Fa

test
0.8684501647949219 0.8893736004829407 	 0.8508411645889282 0.8757103681564331
s_h_high:  0.01563611626625061 s_h_long:  0.01563832002447618 h_f_high:  0.008289068937301636 h_f_long: 
Not Fall
4690
test
0.8698731660842896 0.890015184879303 	 0.8446569442749023 0.8757673501968384
s_h_high:  0.019732028245925903 s_h_long:  0.019737760898281054 h_f_high:  0.0021868348121643066 h_f_long: 
Not Fall
4691
test
0.8636170029640198 0.8860313892364502 	 0.8438252806663513 0.8777682781219482
s_h_high:  0.014027416706085205 s_h_long:  0.01402772646314709 h_f_high:  -0.008361011743545532 h_f_long: 
Not Fall
4692
test
0.8658455610275269 0.8861531615257263 	 0.8447084426879883 0.8795868158340454
s_h_high:  0.013851732015609741 s_h_long:  0.01397740096981511 h_f_high:  -0.005703657865524292 h_f_long: 
Not Fall
4693
test
0.8652364015579224 0.8841361403465271 	 0.8450701832771301 0.8710996508598328
s_h_high:  0.016601353883743286 s_h_long:  0.016670938153066826 h_f_high:  -1.6510486602783203e-05 h_f_

test
0.6148671507835388 0.6189870834350586 	 0.45288991928100586 0.46687501668930054
sideway slight
s_h_high:  0.1570446491241455 s_h_long:  0.15710512786236622 h_f_high:  0.2108367383480072 h_f_long: 
Not Fall
4715
test
0.6146756410598755 0.6183159947395325 	 0.45288705825805664 0.4668356776237488
s_h_high:  0.15663444995880127 s_h_long:  0.1566884043884175 h_f_high:  0.21174612641334534 h_f_long: 
Not Fall
4716
test
0.6144378185272217 0.6172894835472107 	 0.45289742946624756 0.4666934609413147
s_h_high:  0.15606820583343506 s_h_long:  0.15611584212275642 h_f_high:  0.21275374293327332 h_f_long: 
Not Fall
4717
test
0.6143805980682373 0.6167290806770325 	 0.45290401577949524 0.46646741032600403
sideway slight
s_h_high:  0.15586912631988525 s_h_long:  0.15591024258319836 h_f_high:  0.21312832832336426 h_f_long: 
Not Fall
4718
test
0.6143327355384827 0.6163522005081177 	 0.45294371247291565 0.46619975566864014
s_h_high:  0.15577073395252228 s_h_long:  0.1558072600500296 h_f_high:  0.2133